## Build an Agent

<br>

https://python.langchain.com/docs/tutorials/agents/

### Install

In [ ]:
%pip install -U langchain-community langgraph langchain-google-vertexai tavily-python langgraph-checkpoint-sqlite --quiet

### Libs and variables

In [55]:
import os
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

In [56]:
load_dotenv()

True

In [57]:
assert "LANGSMITH_TRACING" in os.environ, "Please set the LANGSMITH_TRACING environment variable."
assert "LANGSMITH_API_KEY" in os.environ, "Please set the LANGSMITH_API_KEY environment variable."
assert "TAVILY_API_KEY" in os.environ, "Please set the TAVILY_API_KEY environment variable."

### Tools

In [58]:
search = TavilySearchResults(max_results=2)
search_results = search.invoke("que horas são em São Paulo")
tools = [search]

### Model

In [72]:
model = init_chat_model("gemini-2.0-flash-001", model_provider="google_vertexai")
response = model.invoke([HumanMessage(content="hi!")])
response.usage_metadata

{'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13}

In [61]:
model_with_tools = model.bind_tools(tools)

In [63]:
response = model_with_tools.invoke([HumanMessage(content="que horas são em São Paulo")])
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: I am sorry, I do not have the current time for São Paulo. I can provide information on current events if you like.
ToolCalls: []


### Agent

In [65]:
agent_executor = create_react_agent(model, tools)

In [66]:
response = agent_executor.invoke({"messages": [HumanMessage(content="que horas são em São Paulo")]})
response["messages"]

[HumanMessage(content='que horas são em São Paulo', additional_kwargs={}, response_metadata={}, id='828bd16c-7ac3-4f82-a69c-eb6c18de52d0'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "hora atual em S\\u00e3o Paulo"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 53, 'candidates_token_count': 14, 'total_token_count': 67, 'prompt_tokens_details': [{'modality': 1, 'token_count': 53}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 14}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.026951187423297336}, id='run-7384a169-dbcc-4051-a9aa-f2729cc98fce-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'hora atual em São Paulo'}, 'id': '53474e5d-8887-44b7-b9ff-fc6f37940dae', 'type': 'tool_call'}], usage_metadata={'input_tokens': 53, 'output_tokens': 14, 'total_tok

In [68]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="que horas são em São Paulo")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

que horas são em São Paulo
================================== Ai Message ==================================

Para saber que horas são em São Paulo, preciso fazer uma pesquisa online. Qual seria uma boa frase de busca para isso?


In [69]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="que horas são em São Paulo")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

No| momento, não consigo fornecer a hora exata em São Paulo.|

### Memory

In [52]:
memory = MemorySaver()

In [70]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [71]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?\n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 66, 'candidates_token_count': 11, 'total_token_count': 77, 'prompt_tokens_details': [{'modality': 1, 'token_count': 66}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 11}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.0005913819609717889}, id='run-7a27443c-4f56-4ff9-998f-212d08f9cdd5-0', usage_metadata={'input_tokens': 66, 'output_tokens': 11, 'total_tokens': 77})]}}
----
